In [9]:
import nltk
import speech_recognition as sr

r = sr.Recognizer()
with sr.AudioFile("joyfully.wav") as source:
    audio = r.record(source)
text = r.recognize_google(audio)
print(text)

the delicious Aroma of freshly baked bread filled the Bakery
